# Raspberry Pi Setup & CAN "Hello World"

## 1. Raspberry Pi Base Setup
Follow the tutorial video: https://www.youtube.com/watch?v=ykTlNf1TXO0  
CAN shield mounting guide: https://wiki.seeedstudio.com/2-Channel-CAN-BUS-FD-Shield-for-Raspberry-Pi/

---

## 2. Activate SSH (recommended)
Working remotely from a faster machine is recommended.
Steps:
- Open terminal (Ctrl + Alt + T)
- Run: `sudo raspi-config`
- Navigate: `3 Interface Options` -> `I1 SSH` -> `Enable`
- After confirmation choose `Finish`
- Show IP address: `hostname -I` (note it)

---

## 3. Create Working Directory & Virtual Environment
Example:
```bash
cd ~/Desktop
mkdir PY
cd PY
python3 -m venv venv        # Choose any name
source venv/bin/activate    # Activate
pip install python-can
# Deactivate later with: deactivate
```
Activation each session:
```bash
cd ~/Desktop/PY
source venv/bin/activate
```
---

## 4. Configure CAN Shield & SPI Overlay
Edit boot config:
```bash
sudo nano /boot/firmware/config.txt
# Add at end:
dtoverlay=seeed-can-fd-hat-v2
```
Save (Ctrl+X, Y) then:
```bash
sudo reboot
dmesg | grep spi     # Check SPI / CAN init messages
ifconfig -a          # can0 and can1 should appear
```
---

## 5. Connect via VS Code Remote SSH
On your PC:
- Open VS Code
- Install extension: Remote - SSH
- Click the >< icon (bottom-left)
- Select: "Connect to Host" -> "Add New SSH Host"
- Enter: `ssh <pi-user>@<pi-ip>` (example: `ssh krya@10.0.0.80`)
- Choose config location (`~/.ssh/config` on your system)
- After "Host added" click "Connect"
- Accept fingerprint
- Enter Pi password (raspberry in this case)
- New VS Code window opens (server installs on first use)
- Status bar shows: `SSH:<ip-address>` (e.g. `SSH:10.0.0.80`)

---

## 6. CAN "Hello World"
Bring up interface:
```bash
sudo ip link set can0 type can bitrate 500000
sudo ip link set can0 up
```

Python example (`hello_can.py`):
```python
import can

def main():
    try:
        bus = can.interface.Bus(channel="can0", interface="socketcan")
        msg = can.Message(
            arbitration_id=0x123,
            data=[0x11, 0x22, 0x33, 0x44, 0x55, 0x66, 0x77, 0x88],
            is_extended_id=False
        )
        bus.send(msg)
        print(f"Message successfully sent on {bus.channel_info}")
        print(f"Sending message with ID 0x{msg.arbitration_id:X} and data {msg.data}")
    except Exception as e:
        print(f"Error: CAN Bus could not be initialized. {e}")
    finally:
        try:
            bus.shutdown()
        except:
            pass

if __name__ == "__main__":
    main()
```

Expected output:
```
Message successfully sent on socketcan channel 'can0'
Sending message with ID 0x123 and data bytearray(b'\x11"3DUfw\x88')
Message sent successfully
```
---

## 7. Optional: Loopback script
- test the loopback script further down to check wether the shield can send and receive on the bus

- in command:
```bash
sudo ip link set can0 down
sudo ip link set can0 type can bitrate 500000 loopback on
sudo ip link set up can0
python loopbacktest.py
```

Expected output:
```
Sending message with ID 0x123 and data bytearray(b'\xde\xad\xbe\xef')
Waiting to receive message...
Received message with ID 0x123 and data bytearray(b'\xde\xad\xbe\xef')
```
---

In [ ]:
# File: send_test.py
import can
import time

# 1. Configure the CAN bus channel
# We use the SocketCAN backend (standard on Linux) and the 'can0' channel.
try:
    bus = can.interface.Bus(channel='can0', bustype='socketcan')
    print(f"CAN bus '{bus.channel}' successfully initialized.")

except Exception as e:
    print(f"ERROR: Could not initialize the CAN bus. Check if can0 is active. Details: {e}")
    exit()

# 2. Create a CAN message
message = can.Message(
    arbitration_id=0x123,  # The CAN ID we are sending
    data=[0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08],  # 8 bytes of payload data
    is_extended_id=False    # Standard ID (11 Bit)
)

# 3. Send the message
print(f"Sending message with ID 0x{message.arbitration_id:X} and data: {message.data.hex().upper()}...")

try:
    bus.send(message)
    print("Message successfully sent!")
except can.CanError:
    print("ERROR: Message could not be sent over the bus.")

# 4. Close the bus
bus.shutdown()

In [ ]:
# Loopback script

import can
import time

# configure CAN bus
bus = can.interface.Bus(channel='can0', interface='socketcan')

# create a message
msg = can.Message(
    arbitration_id=0x123, 
    data=[0xDE, 0xAD, 0xBE, 0xEF], 
    is_extended_id=False
    )

# send the message
print(f"Sending message with ID 0x{msg.arbitration_id:X} and data {msg.data}")
bus.send(msg)
time.sleep(0.1)  # wait a bit for the message to be sent

# receive the message
print("Waiting to receive message...")
received_msg = bus.recv(timeout=1.0)  # wait up to 1 second

bus.shutdown()

if received_msg:
    print(f"Received message with ID 0x{received_msg.arbitration_id:X} and data {received_msg.data}")
else:
    print("No message received")


# (Optional) CAN Sender systemd Service

This guide shows how to run the cyclic CAN sender script (`cantest.py`) automatically at boot on a Raspberry Pi using a custom `systemd` unit. Adjust usernames, paths, and environment names to match your setup.

---
## 1. Prerequisites

Before creating the service unit:

1. CAN shield / interface installed and configured (overlay loaded in `config.txt`).
2. Script file present, e.g.: `/home/<USER>/Desktop/PY/cantest.py`.
3. Python virtual environment created, e.g.:
   ```bash
   cd ~/Desktop
   mkdir PY
   cd PY
   python3 -m venv venv          # or .venv
   source venv/bin/activate
   pip install python-can
   
   ```
4. Manual test succeeds:
   ```bash
   sudo ip link set can0 type can bitrate 500000
   sudo ip link set can0 up
   python cantest.py
   ```

---
## 2. Create the Service File

Edit (with sudo) a new unit file:

```bash
sudo nano /etc/systemd/system/can-sender.service
```

Paste the template below (replace placeholders):

```ini
# /etc/systemd/system/can-sender.service
[Unit]
Description=CAN Bus Cyclic Sender Service
After=network-online.target

[Service]
# IMPORTANT: Replace <USER> with your Raspberry Pi username
User=<USER>
# Adjust WorkingDirectory to the folder containing cantest.py
WorkingDirectory=/home/<USER>/Desktop/PY

# Ensure CAN interface is configured before starting Python script
ExecStartPre=/usr/sbin/ip link set can0 type can bitrate 500000
ExecStartPre=/usr/sbin/ip link set can0 up

# Path to Python interpreter inside your virtual environment
# Use venv OR .venv depending on what you created
ExecStart=/home/<USER>/Desktop/PY/venv/bin/python cantest.py

# Restart policy (always restart on failure or exit)
Restart=always
RestartSec=5

[Install]
WantedBy=multi-user.target
```
Save and exit: `Ctrl + O`, `Enter`, `Ctrl + X`.

---
## 3. Enable and Start the Service

Reload `systemd` so it recognizes the new unit:

```bash
sudo systemctl daemon-reload
```

Enable at boot:

```bash
sudo systemctl enable can-sender.service
```

Start immediately:

```bash
sudo systemctl start can-sender.service
```

Check status:

```bash
sudo systemctl status can-sender.service
```

Follow live logs:

```bash
journalctl -u can-sender.service -f
```

Stop the service:

```bash
sudo systemctl stop can-sender.service
```

Disable autostart:

```bash
sudo systemctl disable can-sender.service
```

---
## 4. Updating the Service

If you change the unit file or move the script / venv:

```bash
sudo nano /etc/systemd/system/can-sender.service   # edit changes
sudo systemctl daemon-reload
sudo systemctl restart can-sender.service
```

---
## 5. Explanation of Key Directives

| Directive | Purpose |
|-----------|---------|
| `After=network-online.target` | Ensures service starts after networking is up (often helpful for remote logging/SSH before script starts). |
| `ExecStartPre=` | Commands run before the main process; here they configure and bring up `can0`. |
| `ExecStart=` | The main process: runs the cyclic sender script via the venv Python. |
| `Restart=always` | Guarantees the service restarts if the script exits or crashes. |
| `RestartSec=5` | Waits 5 seconds between restart attempts. |
| `WantedBy=multi-user.target` | Makes the service part of normal multi-user boot sequence. |

---
## 6. Common Issues & Troubleshooting

| Symptom | Check / Fix |
|---------|-------------|
| `can0` not found | Verify overlay in `/boot/firmware/config.txt` and reboot; run `ip link` to list interfaces. |
| Permission denied | Confirm `User=<USER>` owns the working directory and script. Use `sudo chown -R <USER>:<USER> ~/Desktop/PY`. |
| Python module `can` missing | Activate venv manually and `pip install python-can`; confirm path in `ExecStart`. |
| Wrong bitrate | Adjust `ExecStartPre` line to match actual bus (e.g. `250000`). |
| Service flapping | Inspect logs: `journalctl -u can-sender.service`; add extra logging inside `cantest.py`. |

---
## 7. Minimal `cantest.py` Example

```python
# cantest.py - cyclic sending
import can
import time

# wait time betweeen message sending
INTERVAL = 1.0  

# message definition
message = can.Message(
    arbitration_id=0x123,
    data=[0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08],
    is_extended_id=False
)

print("CAN-Sender started.")
print(f"Send each {INTERVAL} secondes message ID 0x{message.arbitration_id:X}")

try:
    with can.interface.Bus(channel='can0', interface='socketcan') as bus:
        
        while True:
            try:
                # Send message
                bus.send(message)
                print("message sent.") # delete by commenting if log is too packed
                
                # wait on next cycle
                time.sleep(INTERVAL)

            except can.CanError:
                print("Error message couldnt be sent to bus")
                time.sleep(1) # wait in case of error and retry
                
except Exception as e:
    print(f"fatal error when starting bus: {e}")
```

---
## 8. Verifying at Boot

After a reboot:

```bash
systemctl is-active can-sender.service        # should print 'active'
systemctl status can-sender.service           # detailed state
journalctl -u can-sender.service --since "5 minutes ago"
```

If inactive, re-run `daemon-reload`, check paths, and confirm the venv interpreter exists.

---
## 9. Optional Improvements

1. Add an environment file for bitrate/user-configurable parameters.
2. Use `Type=simple` explicitly (default) or `Type=notify` with a small wrapper if later extended.
3. Separate log output using `StandardOutput=journal` and `StandardError=journal` (default) or redirect to a file.

---
## 10. Quick Template With Placeholders

```ini
[Unit]
Description=CAN Cyclic Sender Service
After=network-online.target

[Service]
User=<USER>
WorkingDirectory=/home/<USER>/Desktop/PY
ExecStartPre=/usr/sbin/ip link set can0 type can bitrate <BITRATE>
ExecStartPre=/usr/sbin/ip link set can0 up
ExecStart=/home/<USER>/Desktop/PY/venv/bin/python cantest.py
Restart=always
RestartSec=5

[Install]
WantedBy=multi-user.target
```

Replace `<USER>` and `<BITRATE>` accordingly.

---
### Summary
Once enabled, the service will start automatically at each boot, configure the CAN bus, and run your cyclic sender script continuously.



# CAN specifics explained
(to be continued only in draft mode 14.11.2025)

#### CAN Frame Structure (Extended 29-bit Identifier)

| Field | Length (bits) | Description |
|-------|---------------|-------------|
| Start of Frame (SOF) | 1 | Marks start of frame (dominant) |
| Arbitration Field | 29 | 29-bit Identifier (priority; lower value = higher priority) |
| Control Field | 6 | DLC (4 bits) + reserved bits |
| Data Field | 0–64 | Payload (0–8 bytes for Classical CAN) |
| CRC Field | 16 | Cyclic redundancy check |
| ACK Field | 2 | ACK slot + delimiter |
| End of Frame | 7 | 7 recessive bits marking frame end |

Extended CAN frame example (29-bit ID):
- ID: 0x18FF50E5  
- DLC: 8  
- Data: `[0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08]`

All nodes receive all frames; each decides relevance based on the Identifier.
Lower numeric IDs win bus arbitration (higher priority).
There are no explicit source/destination addresses like in TCP/IP. Frames are broadcast and filtered by ID.

# Example Class
Modelled as dictionary

In [ ]:
class CANMessage:
    def __init__(self, id, data):
        self.id = id
        self.dlc = len(data)
        self.data = data

    def __repr__(self):
        return f"CANMessage(ID=0x{self.id}, DLC={self.dlc}, data={self.data})"

msg = CANMessage(0x123, [0x11, 0x22, 0x33, 0x44])
print(msg)

CANMessage(ID=0x291, DLC=4, data=[17, 34, 51, 68])


### Example BMS

In [11]:
class BMS_Simulator:
    def send_status(self):
        status_msg = CANMessage(0x100, [0x01, 0x02, 0x03, 0x04])
        print(f"Sending: {status_msg}")

class PLC:
    def receive_message(self, msg):
        print(f"Received: {msg}")

bms = BMS_Simulator()
plc = PLC()